# Supermarket Sales Data

Link: https://www.kaggle.com/datasets/yapwh1208/supermarket-sales-data?select=annex4.csv

This is a dataset of sales data of vegetables in a supermarket.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df1 = pd.read_csv('data/annex1.csv')
df2 = pd.read_csv('data/annex2.csv')
df3 = pd.read_csv('data/annex3.csv')
df4 = pd.read_csv('data/annex4.csv')

In [3]:
display(df1.shape)
display(df1.head(3))

display(df2.shape)
display(df2.head(3))

display(df3.shape)
display(df3.head(3))

display(df4.shape)
display(df4.head(3))

(251, 4)

,Item Code,Item Name,Category Code,Category Name
0,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables
1,102900005115199,Sichuan Red Cedar,1011010101,Flower/Leaf Vegetables
2,102900005115625,Local Xiaomao Cabbage,1011010101,Flower/Leaf Vegetables


(878503, 7)

,Date,Time,Item Code,Quantity Sold (kilo),Unit Selling Price (RMB/kg),Sale or Return,Discount (Yes/No)
0,2020-07-01,09:15:07.924,102900005117056,0.396,7.6,sale,No
1,2020-07-01,09:17:27.295,102900005115960,0.849,3.2,sale,No
2,2020-07-01,09:17:33.905,102900005117056,0.409,7.6,sale,No


(55982, 3)

,Date,Item Code,Wholesale Price (RMB/kg)
0,2020-07-01,102900005115762,3.88
1,2020-07-01,102900005115779,6.72
2,2020-07-01,102900005115786,3.19


(251, 3)

,Item Code,Item Name,Loss Rate (%)
0,102900005115168,Niushou Shengcai,4.39
1,102900005115199,Sichuan Red Cedar,10.46
2,102900005115250,Xixia Black Mushroom (1),10.80


In [16]:
len(df2['Item Code'].unique())

246